In [ ]:
# ======================================
# EO-SAR Change Detection (Python version)
# Compatible with Jupyter Notebook
# ======================================

import ee
import geemap

# Authenticate & Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# --------------------------------------
# 1️ Define Area of Interest (replace asset)
# --------------------------------------
aoi = ee.FeatureCollection('projects/lala-285fb/assets/blr_dst')

# Create interactive map
Map = geemap.Map(center=[12.97, 77.59], zoom=10)
Map.addLayer(aoi, {}, 'AOI')

# --------------------------------------
# 2️ Define Pre & Post Dates
# --------------------------------------
pre_start, pre_end = '2018-01-01', '2018-12-31'
post_start, post_end = '2024-01-01', '2024-12-31'

# --------------------------------------
# 3️ Sentinel-2 SR Data
# --------------------------------------
def maskS2(image):
    scl = image.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    return image.updateMask(mask).divide(10000)

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

# --------------------------------------
# 4️ Pre & Post Sentinel-2 Composites
# --------------------------------------
s2_pre = (s2.filterDate(pre_start, pre_end)
             .map(maskS2)
             .median()
             .clip(aoi))

s2_post = (s2.filterDate(post_start, post_end)
              .map(maskS2)
              .median()
              .clip(aoi))

# --------------------------------------
# 5️ Compute Indices
# --------------------------------------
def addIndices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndbi = img.normalizedDifference(['B11', 'B8']).rename('NDBI')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return img.addBands([ndvi, ndbi, ndwi])

s2_pre = addIndices(s2_pre)
s2_post = addIndices(s2_post)

# --------------------------------------
# 6️ Sentinel-1 Pre & Post (VV/VH)
# --------------------------------------
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
        .filterBounds(aoi)
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
        .filter(ee.Filter.eq('resolution_meters', 10))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

s1_pre = s1.filterDate(pre_start, pre_end).median().clip(aoi)
s1_post = s1.filterDate(post_start, post_end).median().clip(aoi)

# --------------------------------------
# 7️ Compute Change Layers (Sentinel-2)
# --------------------------------------
diff = s2_post.select(['NDVI', 'NDBI', 'NDWI']).subtract(
       s2_pre.select(['NDVI', 'NDBI', 'NDWI'])
       ).rename(['NDVI_diff', 'NDBI_diff', 'NDWI_diff'])

# --------------------------------------
# 8️ Thematic Classification
# --------------------------------------
veg_gain = diff.select('NDVI_diff').gt(0.2)
veg_loss = diff.select('NDVI_diff').lt(-0.2)
veg_stable = diff.select('NDVI_diff').gte(-0.2).And(diff.select('NDVI_diff').lte(0.2))
veg_map = veg_gain.multiply(1).add(veg_loss.multiply(2)).add(veg_stable.multiply(3)).rename('Veg_Class')

urb_gain = diff.select('NDBI_diff').gt(0.1)
urb_loss = diff.select('NDBI_diff').lt(-0.1)
urb_stable = diff.select('NDBI_diff').gte(-0.1).And(diff.select('NDBI_diff').lte(0.1))
urb_map = urb_gain.multiply(1).add(urb_loss.multiply(2)).add(urb_stable.multiply(3)).rename('Urban_Class')

wat_gain = diff.select('NDWI_diff').gt(0.15)
wat_loss = diff.select('NDWI_diff').lt(-0.15)
wat_stable = diff.select('NDWI_diff').gte(-0.15).And(diff.select('NDWI_diff').lte(0.15))
wat_map = wat_gain.multiply(1).add(wat_loss.multiply(2)).add(wat_stable.multiply(3)).rename('Water_Class')

# --------------------------------------
# 9️ Confidence Maps
# --------------------------------------
conf_veg = diff.select('NDVI_diff').abs().rename('Veg_Confidence')
conf_urb = diff.select('NDBI_diff').abs().rename('Urban_Confidence')
conf_wat = diff.select('NDWI_diff').abs().rename('Water_Confidence')

# --------------------------------------
# 10 Visualization
# --------------------------------------
Map.addLayer(diff.select('NDVI_diff'), {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}, 'NDVI Change')
Map.addLayer(diff.select('NDBI_diff'), {'min': -0.3, 'max': 0.3, 'palette': ['blue', 'white', 'brown']}, 'NDBI Change')
Map.addLayer(diff.select('NDWI_diff'), {'min': -0.3, 'max': 0.3, 'palette': ['brown', 'white', 'blue']}, 'NDWI Change')

Map.addLayer(veg_map, {'min': 1, 'max': 3, 'palette': ['green', 'red', 'gray']}, 'Vegetation Map')
Map.addLayer(urb_map, {'min': 1, 'max': 3, 'palette': ['orange', 'blue', 'gray']}, 'Urban Map')
Map.addLayer(wat_map, {'min': 1, 'max': 3, 'palette': ['blue', 'brown', 'gray']}, 'Water Map')

Map.addLayer(conf_veg, {'min': 0, 'max': 0.6, 'palette': ['white', 'yellow', 'green']}, 'Vegetation Confidence')
Map.addLayer(conf_urb, {'min': 0, 'max': 0.6, 'palette': ['white', 'yellow', 'orange']}, 'Urban Confidence')
Map.addLayer(conf_wat, {'min': 0, 'max': 0.6, 'palette': ['white', 'yellow', 'blue']}, 'Water Confidence')

Map

# --------------------------------------
# 1️1️ Export to Local Directory (TIFF)
# --------------------------------------
output_dir = "C:\Users\USER\Downloads\Galaxeye-assig\Assig-2"

geemap.ee_export_image(s2_pre, filename=f"{output_dir}/S2_PreComposite.tif", scale=10, region=aoi.geometry(), file_per_band=False)
geemap.ee_export_image(s2_post, filename=f"{output_dir}/S2_PostComposite.tif", scale=10, region=aoi.geometry(), file_per_band=False)
geemap.ee_export_image(s1_pre, filename=f"{output_dir}/S1_PreComposite.tif", scale=10, region=aoi.geometry(), file_per_band=False)
geemap.ee_export_image(s1_post, filename=f"{output_dir}/S1_PostComposite.tif", scale=10, region=aoi.geometry(), file_per_band=False)

geemap.ee_export_image(veg_map, filename=f"{output_dir}/Vegetation_Change_Thematic.tif", scale=10, region=aoi.geometry())
geemap.ee_export_image(conf_veg, filename=f"{output_dir}/Vegetation_Confidence.tif", scale=10, region=aoi.geometry())
geemap.ee_export_image(urb_map, filename=f"{output_dir}/Urban_Change_Thematic.tif", scale=10, region=aoi.geometry())
geemap.ee_export_image(conf_urb, filename=f"{output_dir}/Urban_Confidence.tif", scale=10, region=aoi.geometry())
geemap.ee_export_image(wat_map, filename=f"{output_dir}/Water_Change_Thematic.tif", scale=10, region=aoi.geometry())
geemap.ee_export_image(conf_wat, filename=f"{output_dir}/Water_Confidence.tif", scale=10, region=aoi.geometry())

print("✅ All composites and change detection maps exported successfully!")
